In [1]:
! pip install ultralytics boto3


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
from ultralytics import YOLO
import os
import yaml
import boto3

In [ ]:
! tar -xzf dataset-full.tar.gz

In [3]:
EPOCHS=os.environ.get("EPOCHS", 1)
IMGSZ=640
DATA_CONFIG_PATH = "utils/data.yaml"
LABEL_NAME = os.environ.get("LABEL_NAME", "mylabel")

EXPORT_BUCKET = "model-zoo"
MODEL_PATH = "models/model.onnx"

AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")
AWS_S3_BUCKET = os.environ.get("AWS_S3_BUCKET")
AWS_S3_ENDPOINT = os.environ.get("AWS_S3_ENDPOINT")

In [4]:
with open(DATA_CONFIG_PATH, 'r') as f:
    data_config = yaml.safe_load(f)
data_config['names'][0] = LABEL_NAME
with open(DATA_CONFIG_PATH, 'w') as f:
    data_config = yaml.dump(data_config, f)

In [ ]:
! ls

In [5]:
model = YOLO('yolov8n.pt')
results = model.train(data=DATA_CONFIG_PATH, epochs=EPOCHS, imgsz=IMGSZ)

Ultralytics YOLOv8.1.17 🚀 Python-3.9.16 torch-2.2.1+cu121 CPU (AMD EPYC 7763 64-Core Processor)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=utils/data.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_box

[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


Model summary: 225 layers, 3011043 parameters, 3011027 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'


train: Scanning /opt/app-root/src/dataset/labels/train.cache... 36 images, 1 backgrounds, 0 corrupt: 100%|██████████| 37/37 [00:00<?, ?it/s]
val: Scanning /opt/app-root/src/dataset/labels/val.cache... 6 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6/6 [00:00<?, ?it/s]


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G     0.6284      4.175     0.9239         11        640: 100%|██████████| 3/3 [00:50<00:00, 16.90s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:04<00:00,  4.21s/it]


                   all          6          6    0.00222      0.667    0.00462    0.00444

1 epochs completed in 0.017 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.17 🚀 Python-3.9.16 torch-2.2.1+cu121 CPU (AMD EPYC 7763 64-Core Processor)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.70s/it]


                   all          6          6    0.00222      0.667    0.00468    0.00449
Speed: 0.3ms preprocess, 243.2ms inference, 0.0ms loss, 16.5ms postprocess per image
Results saved to runs/detect/train


In [6]:
model = YOLO('runs/detect/train/weights/best.pt')

In [7]:
model.export(format="onnx")

Ultralytics YOLOv8.1.17 🚀 Python-3.9.16 torch-2.2.1+cu121 CPU (AMD EPYC 7763 64-Core Processor)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (5.9 MB)

ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: export success ✅ 2.4s, saved as 'runs/detect/train/weights/best.onnx' (11.7 MB)

Export complete (8.9s)
Results saved to /opt/app-root/src/runs/detect/train/weights
Predict:         yolo predict task=detect model=runs/detect/train/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=runs/detect/train/weights/best.onnx imgsz=640 data=utils/data.yaml  
Visualize:       https://netron.app


'runs/detect/train/weights/best.onnx'

In [8]:
model_path = "runs/detect/train/weights/best.onnx"

In [9]:
session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)
client = session.client("s3", endpoint_url=AWS_S3_ENDPOINT)

In [10]:
client.upload_file(model_path, EXPORT_BUCKET, MODEL_PATH)